In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import os

#Create an empty dataframe
df_all = pd.DataFrame()

for root, dirs, files in os.walk("../../data/csv"):
    for ctuName in files:
        if(ctuName.find(".gz")+1 or ctuName.find('capture20110817')+1):
            continue

        df=pd.read_csv('../../data/csv/'+ctuName)
        #BC already normalized
        #Normalize ID, OD, IDW, ODW, AC
        for col in ['ID','OD','IDW','ODW','AC']:
            df[col]=(df[col]-df[col].min())/(df[col].max()-df[col].min())
        #Add df values to df_all
        df_all = df_all.append(df)

In [2]:
#Get df_all size
df_all.shape
#We are getting 2.874.213 rows instead of 2.887.787(paper). This is because we dropped some comunications in different protocols. 

(2874213, 9)

In [3]:
#Get dataframe column labels
features=df_all.columns.values
X = np.array(df_all[features[1:8]])
y = np.array(df_all[features[8]])



In [4]:
#Run kmeans
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
labels = kmeans.predict(X)


In [5]:
#Calcular la moda de labels
mode_labels=np.bincount(labels).argmax()
print(mode_labels)
#Contar la cantidad de veces que aparece la moda
count_mode_labels=np.bincount(labels).max()
count_mode_labels


0


1115561

In [6]:
#Create an array of numbers from 2 to 15 squared
ar=np.arange(2,16)
ar=ar**2
for i in ar:
    print(i)

4
9
16
25
36
49
64
81
100
121
144
169
196
225
